# Taking vacations on the best possible weather conditions

## Dependecies and configuration

In [16]:
# Import libraries
import pandas as pd
import requests
import os
import gmaps

# Import keys.
from api_keys import g_key

# Weather information for 1000 cities dataset file
one_thousand_cities_weather_data_path = os.path.join("output", "one_thousand_unique_cities.csv")

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Base query for Goodle nearbysearch API.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Load datasets and prepare parameters

In [17]:
# Read weather information for 1000 cities dataset.
cities_df = pd.read_csv(one_thousand_cities_weather_data_path)

# Create lists' parameters for google maps
locations = cities_df[["lat", "lon"]].astype(float)
weather_humidity = cities_df["humidity"].astype(float)


## Create map to show humidity in a 1000 cities

In [18]:
# Create a heat map showing the humidity for a list of cities

humidity_fig = gmaps.figure()

humidity_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

humidity_fig.add_layer(humidity_heat_layer)

humidity_fig


Figure(layout=FigureLayout(height='420px'))

## Look up the cities with the best weather

In [20]:
# Look up cities in the dataframe according to certain conditions

best_weather_conditions_cities = cities_df[
    (cities_df['temperature'] > 65) & (cities_df['temperature'] < 75)
    & (cities_df['wind_speed'] < 10)
    & (cities_df['cloudiness'] == 0)
]

best_weather_conditions_cities = best_weather_conditions_cities.reset_index(drop = True)
best_weather_conditions_cities

,city_id,city_name,state,country,lat,lon,temperature,humidity,cloudiness,wind_speed,weather_condition,weather_condition_desc
0,1811276,Fengming,NaN,CN,34.441391,107.617783,71.55,53.0,0.0,1.39,Clear,clear sky
1,1138565,Kalāt,NaN,AF,34.389931,63.377800,68.85,24.0,0.0,5.59,Clear,clear sky
2,6540699,Nocera Superiore,NaN,IT,40.742111,14.672940,74.52,66.0,0.0,1.01,Clear,clear sky
3,1788830,Xinji,NaN,CN,33.789299,113.000252,73.36,69.0,0.0,5.50,Clear,clear sky
4,1787989,Xuelu,NaN,CN,34.409611,108.340523,73.00,83.0,0.0,4.47,Clear,clear sky
5,3183388,Aiello del Sabato,NaN,IT,40.889118,14.820840,72.82,67.0,0.0,7.05,Clear,clear sky
6,3459181,Lagoa Formosa,NaN,BR,-18.778610,-46.407501,73.00,65.0,0.0,4.00,Clear,clear sky
7,3461200,Inhomirim,NaN,BR,-22.569719,-43.185001,73.17,69.0,0.0,8.05,Clear,clear sky
8,6321162,Belo Horizonte,NaN,BR,-19.926229,-43.939819,72.30,46.0,0.0,9.17,Clear,clear sky
9,3870243,Taltal,NaN,CL,-25.400000,-70.483330,69.94,39.0,0.0,4.68,Clear,clear sky


## Look up hotels near the cities with the best weather

In [21]:
# Interact with Google API to get data on one hotel close to each city in the list of cities with ideal weather.

# Fix parameter values to call the API
target_search = "hotel"
target_type = "lodging"
target_radius = 5000

# Parameters for the query to google nearbysearch API.
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

counter = 0

# Call the google API for each city with ideal weather conditions.
print("Starting data request...")

for index, city in best_weather_conditions_cities.iterrows() :
    counter += 1
    
    target_coordinates = f"{city['lat']},{city['lon']}"
    params.update({"location": target_coordinates})
    
    print(f"Requesting data for: {counter} - {city['city_name']}, {city['country']} (lat: {city['lat']}, lon: {city['lon']})...")
    try :
        response = requests.get(base_url, params=params)
        hotels_data = response.json()
    
        # Only get data from the cities that had a hotel within 5000
        if hotels_data['status'] == "OK" :
            best_weather_conditions_cities.loc[index, 'hotel_name'] = hotels_data['results'][0]['name']
            best_weather_conditions_cities.loc[index, 'hotel_vicinity'] = hotels_data['results'][0]['vicinity']
            print("Data received.")
        else :
            print("Data not available.")
    except:
        print(f"There was an error retrieving the hotel information for this city: {city['city_id']} - {city['city_name']}, {city['country']}")

print("Data request completed.")

# Remove all cities that didn't have a hotel within 5000 meters.
best_weather_conditions_cities.dropna(subset = ['hotel_name'], inplace = True)

Starting data request...
Requesting data for: 1 - Fengming, CN (lat: 34.441390999999996, lon: 107.617783)...
Data received.
Requesting data for: 2 - Kalāt, AF (lat: 34.389931, lon: 63.3778)...
Data not available.
Requesting data for: 3 - Nocera Superiore, IT (lat: 40.742111, lon: 14.672939999999999)...
Data received.
Requesting data for: 4 - Xinji, CN (lat: 33.789299, lon: 113.000252)...
Data not available.
Requesting data for: 5 - Xuelu, CN (lat: 34.409611, lon: 108.340523)...
Data not available.
Requesting data for: 6 - Aiello del Sabato, IT (lat: 40.889117999999996, lon: 14.820839999999999)...
Data received.
Requesting data for: 7 - Lagoa Formosa, BR (lat: -18.77861, lon: -46.407501)...
Data received.
Requesting data for: 8 - Inhomirim, BR (lat: -22.569719, lon: -43.185001)...
Data received.
Requesting data for: 9 - Belo Horizonte, BR (lat: -19.926229, lon: -43.939819)...
Data received.
Requesting data for: 10 - Taltal, CL (lat: -25.4, lon: -70.48333000000001)...
Data received.
Requ

In [22]:
locations = best_weather_conditions_cities[["lat", "lon"]].astype(float)
weather_humidity = best_weather_conditions_cities["humidity"].astype(float)
maps_info_box = [
    f"<B>Hotel name</B> <BR> {city['hotel_name']} <BR> \
    <B>City</B> <BR> {city['city_name']} <BR> \
    <B>Country</B> <BR> {city['country']} <BR> \
    <B>Weather conditions</B> <BR> {city['weather_condition_desc']}" \
    for index, city in best_weather_conditions_cities.iterrows()
]

hotels_fig = gmaps.figure()

hotels_heat_layer = gmaps.heatmap_layer(
    locations,
    weights = weather_humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)

hotels_markers = gmaps.marker_layer(
        locations,
        info_box_content = maps_info_box
)

hotels_fig.add_layer(hotels_heat_layer)
hotels_fig.add_layer(hotels_markers)

hotels_fig

Figure(layout=FigureLayout(height='420px'))